# Custom hot pixel filtering
Used to preprocess evaluation dataset.

Instead of using default hot pixel filtering in Steinbock  pipeline, this script was used. 

Outout of this script is filtered images in tifffile and a csv file summarising the removed hot pixels.

Transfer the filtered images into Steinbock img/ folder and continue with the next steps in the Steinbock pipline.  (e.g. calculating cell masks)

## Improt libraries

In [3]:
import anndata as ad
from pathlib import Path
import os
import numpy as np
import pandas as pd
import errno
import glob
import skimage
import matplotlib.pyplot as plt
from scipy.ndimage import maximum_filter, minimum_filter
import tifffile

/home/ubuntu/miniconda3/envs/test_cisi_imc_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Specify input and output folder

In [6]:
# path to data
input_path = Path('/mnt/projects/data/publication_2/0_preprocess_evaluationdataset')

# path to raw images data
img_paths = glob.glob(os.path.join(input_path,'img_original/*.tiff'))

# set output dir
out_path = Path(os.path.join(input_path, 'steinbock_cthpf/img'))
# Create output directory if it doesn't exist
out_path.mkdir(parents=True, exist_ok=True)

## Define custom hot pixel filtering method
for each hot pixel detected, take min value from surrounging pixel and propagete the value to self and surrounding pixels.

In [7]:
def custom_hbpf(matrix_cxy,c,x,y,v):
    matrix_cxy[c,x,y] = v
    matrix_cxy[c,x+1,y] = v
    matrix_cxy[c,x-1,y] = v
    matrix_cxy[c,x,y+1] = v
    matrix_cxy[c,x+1,y+1] = v
    matrix_cxy[c,x-1,y+1] = v
    matrix_cxy[c,x,y-1] = v
    matrix_cxy[c,x+1,y-1] = v
    matrix_cxy[c,x-1,y-1] = v
    return matrix_cxy

## Detect hot pixels and perform custom filtering
A pixel was defined as a hot pixel if 1. it was brighter than the brightest surrounding pixels by more than 100 counts or 2. it was brighter than 500 counts. After custom hot pixel filtering, standard hot pixel filtering (sane as the default Steinbock filtering) was also performed.

In [8]:
# set threshold (count) for hot pixel and bright pixel
hpthd = 100 # (signed) difference in count value compared to max of surrounding pixel
bpthd = 500 # raw count value
# set kernel for surrounding pixel (excluding self)
kernel = np.ones((1, 3, 3), dtype=bool)
kernel[0, 1, 1] = False
# list for storing hotpixels
hbplist = np.empty((6,0))

for img_id, ip in enumerate(img_paths):
    # read images
    ti = tifffile.imread(ip)
    # remove Extra markers (PankCK:Lu175,Ir191,Ir193) for hpf (will be concatenated after filtering)
    img = ti[:-3,:,:] 
    nuc_img = ti[-3:,:,:]
    # calculate maximum value of surrounding pixel (excluding self) for each pixel
    max_neighbor_img = maximum_filter(img, footprint=kernel, mode="mirror")
    # caluclate minimum value of surrounding pixel as a repolacement value
    min_neighbor_img = minimum_filter(img, footprint=kernel, mode="mirror")
    # find hot/bright pixels (hbp) and get color, x, y, value of them
    cxy = np.array(np.nonzero((img - max_neighbor_img > hpthd)|(img > bpthd)))
    cxyv = np.append(cxy,np.array([img[cxy[0],cxy[1],cxy[2]]],dtype = np.int32),axis = 0)
    cxyv = cxyv[:,cxyv[-1,:].argsort()[::-1]] # sorted from brightest  
    # create edge extended image for custom filtering
    tmp_img = np.append(img, img[:,[-1],:], axis = 1) # add dummy row
    tmp_img = np.append(tmp_img, tmp_img[:,:,[-1]], axis = 2) # add dummy col
    # for each hbp, take min value from surrounging pixel and propagete the value to self and surrounding pixels
    for cur_cxyv in cxyv.T: 
        tmp_img = custom_hbpf(tmp_img, cur_cxyv[0],cur_cxyv[1],cur_cxyv[2],
                              min_neighbor_img[cur_cxyv[0],cur_cxyv[1],cur_cxyv[2]])

    # perform standard hot pixel filtering (thd = 50, taking the max of neighboring pixel) 
    # plus remove dummy edges and save 
    fimg = np.where(tmp_img[:,:-1,:-1] - max_neighbor_img > 50, max_neighbor_img, tmp_img[:,:-1,:-1])
    
    # save filtered images      
    tifffile.imsave(os.path.join(out_path,ip.split("/")[-1]), np.append(fimg, nuc_img, axis = 0) )
    
    # store hbp data into an array
    hbplist = np.hstack((hbplist,np.vstack((cxyv,min_neighbor_img[cxyv[0],cxyv[1],cxyv[2]], np.repeat(img_id,cxyv.shape[1])))))
    

## Save csv 
list of hot pixels and the corresponding value, with which surrounding pixels inclusing self were replaced.

In [9]:
pd.DataFrame(hbplist.T, columns = ['channel','x','y','value_original','value_filtered','img_id']).to_csv(
    os.path.join(out_path,'..','hot_bright_pixels.csv'))